In [ ]:
#Loding libraries

In [2]:
import yfinance  as yf
import datetime as dt
import datetime as dt
import numpy as np
import pandas as pd

In [ ]:
# give the date between when to be start,when to be end and write the whitch stock you want

In [3]:
start=dt.datetime(2022,7,1)
end=dt.datetime(2023,7,28)
symbol='SBIN.NS'

In [ ]:
#loading  data

In [5]:
stk_data = yf.download(symbol, start=start, end=end)

[*********************100%%**********************]  1 of 1 completed


In [ ]:
# choose which cloumns you want

In [19]:
stk_data=stk_data[["Open","High","Low","Close"]]

In [ ]:
# preproceesing the data

In [20]:
from sklearn.preprocessing import MinMaxScaler
Ms = MinMaxScaler()
data1= Ms.fit_transform(stk_data)
print("Len:",data1.shape)

Len: (266, 4)


In [21]:
data1=pd.DataFrame(data1,columns=["Open","High","Low","Close"])

In [ ]:
# define training size

In [22]:
training_size=round(len(data1)*0.80)

In [23]:
print(training_size)

213


In [ ]:
#split input and output

In [24]:
X_train=data1[:training_size]
X_test=data1[training_size:]
y_train=data1[:training_size]
y_test=data1[training_size:]

In [25]:
print("X_train length:",X_train.shape)
print("X_test length:",X_test.shape)
print("y_train length:",y_train.shape)
print("y_test length:",y_test.shape)

X_train length: (213, 4)
X_test length: (53, 4)
y_train length: (213, 4)
y_test length: (53, 4)


In [26]:
listt=["Close","High","Open","Low"]

In [27]:
datasetTwo=data1[listt]

In [ ]:
# give the  Number of observations to use for testing and split it

In [28]:
test_obs = 28
train =datasetTwo[:-test_obs]
test = datasetTwo[-test_obs:]
from statsmodels.tsa.api import VAR

In [ ]:
# print aic and bic metrics for model selection lowest aic,bic values indicates best fitting model

In [29]:
for i in [1,2,3,4,5,6,7,8,9,10]:
        model = VAR(train)
        results = model.fit(i)
        print('Order =', i)
        print('AIC: ', results.aic)
        print('BIC: ', results.bic)
        print()

Order = 1
AIC:  -29.567996965119015
BIC:  -29.275333662069638

Order = 2
AIC:  -29.571031135744118
BIC:  -29.042650012943604

Order = 3
AIC:  -29.602543294674675
BIC:  -28.837017989417244

Order = 4
AIC:  -29.497354163233517
BIC:  -28.493243753642393

Order = 5
AIC:  -29.401051596499467
BIC:  -28.15690039435561

Order = 6
AIC:  -29.35711499343414
BIC:  -27.871452333233147

Order = 7
AIC:  -29.268735967059524
BIC:  -27.540075991213083

Order = 8
AIC:  -29.202251023526628
BIC:  -27.22909246362288

Order = 9
AIC:  -29.126662467933333
BIC:  -26.907488421968146

Order = 10
AIC:  -29.078537177305414
BIC:  -26.611814882794324



In [ ]:
# select_order method is used to automate the process of choosing the lag order based on the specified information criterion

In [30]:
x = model.select_order(maxlags=12)
order=x.selected_orders["aic"]
result = model.fit(order)

In [ ]:
# make prediction

In [31]:
lagged_Values = train.values[-order:]
pred = result.forecast(y=lagged_Values,steps=28) 
preds=pd.DataFrame(pred,columns=listt)
preds.to_csv("varforecasted_{}.csv".format(test_obs))

In [ ]:
#evaution metrics

In [32]:
from sklearn.metrics import mean_squared_error
rmse= round(mean_squared_error(test,pred,squared=False))
from sklearn.metrics import mean_absolute_percentage_error
mape=mean_absolute_percentage_error(test,pred)

In [34]:
performance={"Model":[],"RMSE":[],"MaPe":[],"Lag":[],"Test":[]}
performance["Model"].append(listt)
performance["RMSE"].append(rmse)
performance["MaPe"].append(mape)
performance["Lag"].append(order)
performance["Test"].append(test_obs)
perf=pd.DataFrame(performance)

In [35]:
data1

,Open,High,Low,Close
0,0.000000,0.000000,0.000000,0.000000
1,0.021578,0.033282,0.040140,0.041601
2,0.072750,0.069673,0.066582,0.033407
3,0.072750,0.075894,0.085059,0.082572
4,0.127312,0.120995,0.134119,0.125433
...,...,...,...,...
261,0.904131,0.937481,0.945843,0.934447
262,0.935265,0.952411,0.979611,0.950520
263,0.976880,0.951788,0.915260,0.899779
264,0.916461,0.917885,0.946799,0.934762


In [36]:
perf

,Model,RMSE,MaPe,Lag,Test
0,"[Close, High, Open, Low]",0,0.162934,3,28
